In [1]:
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import librosa
from matplotlib import cm
from librosa import display
import pylab
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank

In [2]:
audio_genres = pd.read_csv("audiofile_genres.csv", sep=";")
audio_genres.head()

,Audio_Filenames,track_genre_1,track_genre_2
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music


In [3]:
occur_col1 = audio_genres["track_genre_1"].value_counts()
occur_col1

Atmosphere            8992
Filmscore             8305
Rock                  4405
Electronica           3356
Hip Hop, Rap          2364
Country, Folk         2305
Pop                   2042
Classical Music       1724
Easy Listening        1620
World Music           1573
Ambient, Chill        1503
Dance                 1394
Indie, Alternative    1153
Funk, Soul            1069
Jazz                  1005
House                  761
Latin                  524
Techno                 375
Big Band               371
Blues                  336
60ies                  333
Drum'n'Bass            327
Military               262
R'n'B                  200
German Folk            189
80ies                  181
Historical Music       169
Swing                  127
Sound Design           126
Christmas              104
Acoustic               100
Orchestral              95
Hard, Heavy             81
Kids                    78
70ies                   75
Religious Music         73
Reggae, Ska             63
T

In [4]:
occur_col2 = audio_genres["track_genre_2"].value_counts()
occur_col2

Pop                   4437
Filmscore             4425
Electronica           3624
Ambient, Chill        3100
Classical Music       1381
World Music           1341
Indie, Alternative     688
Orchestral             655
Kids                   628
Trailer                612
Acoustic               600
Hard, Heavy            555
Sound Design           520
Jazz                   503
Christmas              409
Dance                  386
Drone                  369
Blues                  352
Swing                  331
Easy Listening         311
Piano                  296
Atmosphere             293
R'n'B                  292
70ies                  272
Funk, Soul             228
Song                   223
Dubstep                221
Rock'n'Roll            206
80ies                  189
Religious Music        182
Pizzicato              180
Percussion             155
Punk                   147
60ies                  144
Reggae, Ska            134
House                  128
90ies                  122
H

In [5]:
occur_tot = occur_col1 + occur_col2
occur_tot.sort_values(ascending=False)

Filmscore             12730.0
Atmosphere             9285.0
Electronica            6980.0
Pop                    6479.0
Ambient, Chill         4603.0
Rock                   4469.0
Classical Music        3105.0
World Music            2914.0
Hip Hop, Rap           2433.0
Country, Folk          2413.0
Easy Listening         1931.0
Indie, Alternative     1841.0
Dance                  1780.0
Jazz                   1508.0
Funk, Soul             1297.0
House                   889.0
Orchestral              750.0
Kids                    706.0
Acoustic                700.0
Blues                   688.0
Trailer                 657.0
Sound Design            646.0
Hard, Heavy             636.0
Latin                   572.0
Christmas               513.0
R'n'B                   492.0
60ies                   477.0
Swing                   458.0
Drone                   412.0
Big Band                403.0
Techno                  388.0
80ies                   370.0
Drum'n'Bass             352.0
70ies     

In [6]:
audio_genres["track_genre_1"].replace({"Big Band": "Others", "German Folk": "Others", "Techno": "Others", "80ies": "Others", "Drum'n'Bass": "Others", "70ies": "Others", "Piano": "Others", "Historical Music": "Others", "Military": "Others", "Dubstep": "Others", "Religious Music": "Others", "Rock'n'Roll": "Others", "Song": "Others", "Pizzicato": "Others", "Percussion": "Others", "German, Folk": "Others", "Reggae, Ska": "Others", "Punk": "Others", "90ies": "Others", "Disco": "Others", "Industrial": "Others", "20ies": "Others", "Breakbeat": "Others", "A Cappella": "Others", "50ies": "Others", "40ies": "Others", "30ies": "Others"}, inplace=True)
newoccur_col1 = audio_genres["track_genre_1"].value_counts()
newoccur_col1

Atmosphere            8992
Filmscore             8305
Rock                  4405
Electronica           3356
Others                2457
Hip Hop, Rap          2364
Country, Folk         2305
Pop                   2042
Classical Music       1724
Easy Listening        1620
World Music           1573
Ambient, Chill        1503
Dance                 1394
Indie, Alternative    1153
Funk, Soul            1069
Jazz                  1005
House                  761
Latin                  524
Blues                  336
60ies                  333
R'n'B                  200
Swing                  127
Sound Design           126
Christmas              104
Acoustic               100
Orchestral              95
Hard, Heavy             81
Kids                    78
Trailer                 45
Drone                   43
Name: track_genre_1, dtype: int64

In [7]:
audio_genres["track_genre_2"].replace({"Big Band": "Others", "German Folk": "Others", "Techno": "Others", "80ies": "Others", "Drum'n'Bass": "Others", "70ies": "Others", "Piano": "Others", "Historical Music": "Others", "Military": "Others", "Dubstep": "Others", "Religious Music": "Others", "Rock'n'Roll": "Others", "Song": "Others", "Pizzicato": "Others", "Percussion": "Others", "German, Folk": "Others", "Reggae, Ska": "Others", "Punk": "Others", "90ies": "Others", "Disco": "Others", "Industrial": "Others", "20ies": "Others", "Breakbeat": "Others", "A Cappella": "Others", "50ies": "Others", "40ies": "Others", "30ies": "Others"}, inplace=True)
newoccur_col2 = audio_genres["track_genre_2"].value_counts()
newoccur_col2

Pop                   4437
Filmscore             4425
Electronica           3624
Ambient, Chill        3100
Others                2951
Classical Music       1381
World Music           1341
Indie, Alternative     688
Orchestral             655
Kids                   628
Trailer                612
Acoustic               600
Hard, Heavy            555
Sound Design           520
Jazz                   503
Christmas              409
Dance                  386
Drone                  369
Blues                  352
Swing                  331
Easy Listening         311
Atmosphere             293
R'n'B                  292
Funk, Soul             228
60ies                  144
House                  128
Country, Folk          108
Hip Hop, Rap            69
Rock                    64
Latin                   48
Name: track_genre_2, dtype: int64

In [8]:
newoccur_tot = newoccur_col1 + newoccur_col2
newoccur_tot.sort_values(ascending=False)

Filmscore             12730
Atmosphere             9285
Electronica            6980
Pop                    6479
Others                 5408
Ambient, Chill         4603
Rock                   4469
Classical Music        3105
World Music            2914
Hip Hop, Rap           2433
Country, Folk          2413
Easy Listening         1931
Indie, Alternative     1841
Dance                  1780
Jazz                   1508
Funk, Soul             1297
House                   889
Orchestral              750
Kids                    706
Acoustic                700
Blues                   688
Trailer                 657
Sound Design            646
Hard, Heavy             636
Latin                   572
Christmas               513
R'n'B                   492
60ies                   477
Swing                   458
Drone                   412
dtype: int64

In [9]:
audio_genres.set_index('Audio_Filenames', inplace=True)

In [10]:
for f in audio_genres.index:
    rate, signal = wavfile.read('wav/'+f)
    audio_genres.at[f, 'length'] = signal.shape[0]/rate

In [12]:
audio_genres.sort_values('length',ascending=True)

,track_genre_1,track_genre_2,length
Audio_Filenames,,,
ivox117_19_The-Silk-Road_026966.wav,World Music,Electronica,0.000000
ivox117_01_Yangtze-River_026965.wav,World Music,Pop,0.000000
ivox117_14_Yunnan_026963.wav,World Music,Pop,0.048000
wpm037_55_Slam-20_721277.wav,Sound Design,NaN,0.899660
wpm037_34_Noise-34_721256.wav,Sound Design,NaN,1.053991
...,...,...,...
ivox013_07_Mystic-Dance_000469.wav,Jazz,NaN,563.533333
bcp022_17_Phoenix_728336.wav,Dance,Electronica,572.786939
bcp022_10_Put-Your-Hands-Up_728329.wav,Dance,Electronica,579.422041


In [13]:
audio_genres = audio_genres.drop(audio_genres[audio_genres.length < 40].index)

In [14]:
audio_genres = audio_genres.reset_index('Audio_Filenames')

In [15]:
audio_genres.sort_values('length',ascending=True)

,Audio_Filenames,track_genre_1,track_genre_2,length
2734,4em096_09_A-Time-To-Play---w--Kit_706535.wav,Rock,NaN,40.000000
12391,din027_31_Special-In-Aisle-Twelve_718161.wav,Easy Listening,Filmscore,40.007982
20120,ivox035_27_In-The-Blue-Hour-Shortcut_018035.wav,Filmscore,NaN,40.040000
37907,rsm173_151_From-One-Caper-To-The-Next_736757.wav,World Music,NaN,40.044989
20377,ivox046_29_Marias_001526.wav,Classical Music,Orchestral,40.045714
...,...,...,...,...
19559,ivox013_07_Mystic-Dance_000469.wav,Jazz,NaN,563.533333
6410,bcp022_17_Phoenix_728336.wav,Dance,Electronica,572.786939
6403,bcp022_10_Put-Your-Hands-Up_728329.wav,Dance,Electronica,579.422041
6292,bcp015_04_Vibrations-Rising_721800.wav,Atmosphere,"Ambient, Chill",729.186259


In [16]:
two_columns = [audio_genres["track_genre_1"], audio_genres["track_genre_2"]]
headers = ["genre_1", "genre_2"]
two_columns = pd.concat(two_columns, axis=1, keys=headers)
two_columns

,genre_1,genre_2
0,Atmosphere,World Music
1,Atmosphere,World Music
2,Atmosphere,World Music
3,Atmosphere,World Music
4,Atmosphere,World Music
...,...,...
42769,Rock,NaN
42770,Rock,NaN
42771,Rock,NaN
42772,Rock,NaN


In [17]:
two_columns['genre_2'].fillna('', inplace=True)
two_columns

,genre_1,genre_2
0,Atmosphere,World Music
1,Atmosphere,World Music
2,Atmosphere,World Music
3,Atmosphere,World Music
4,Atmosphere,World Music
...,...,...
42769,Rock,
42770,Rock,
42771,Rock,
42772,Rock,


In [18]:
two_columns.values.sort()
two_columns

,genre_1,genre_2
0,Atmosphere,World Music
1,Atmosphere,World Music
2,Atmosphere,World Music
3,Atmosphere,World Music
4,Atmosphere,World Music
...,...,...
42769,,Rock
42770,,Rock
42771,,Rock
42772,,Rock


In [19]:
trackgenre1 = two_columns["genre_1"]
trackgenre2 = two_columns["genre_2"]
audio_genres['multi_genres'] = trackgenre1 + ' ' + trackgenre2

In [20]:
pd.set_option('display.max_rows', 450)
audio_genres['multi_genres'].value_counts()

 Filmscore                            3094
Atmosphere Filmscore                  2391
 Atmosphere                           2008
 Electronica                          1387
 Rock                                  937
Atmosphere Electronica                 857
 Pop                                   839
 Classical Music                       814
Ambient, Chill Filmscore               787
 World Music                           777
 Others                                775
 Hip Hop, Rap                          743
Electronica Filmscore                  719
Filmscore Others                       662
Ambient, Chill Atmosphere              653
Pop Rock                               637
 Ambient, Chill                        613
 Country, Folk                         608
Filmscore Pop                          601
 Indie, Alternative                    525
 Dance                                 511
Electronica Pop                        469
 Easy Listening                        460
Ambient, Ch

In [21]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Filmscore'].index, size=2750, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
40019,wpm100_02_Drive-This-Car_745440.wav,Rock,NaN,144.355442,Rock
40020,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
40021,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock
40022,wpm100_05_Time-Moves-On_745443.wav,Rock,NaN,121.190295,Rock


In [22]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Atmosphere Filmscore'].index, size=2050, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
37969,wpm100_02_Drive-This-Car_745440.wav,Rock,NaN,144.355442,Rock
37970,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
37971,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock
37972,wpm100_05_Time-Moves-On_745443.wav,Rock,NaN,121.190295,Rock


In [23]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Atmosphere'].index, size=1600, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
36369,wpm100_02_Drive-This-Car_745440.wav,Rock,NaN,144.355442,Rock
36370,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
36371,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock
36372,wpm100_05_Time-Moves-On_745443.wav,Rock,NaN,121.190295,Rock


In [24]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Electronica'].index, size=1000, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
35369,wpm100_02_Drive-This-Car_745440.wav,Rock,NaN,144.355442,Rock
35370,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
35371,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock
35372,wpm100_05_Time-Moves-On_745443.wav,Rock,NaN,121.190295,Rock


In [25]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Rock'].index, size=500, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
34869,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
34870,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
34871,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
34872,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [26]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Atmosphere Electronica'].index, size=460, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
34409,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
34410,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
34411,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
34412,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [27]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Pop'].index, size=440, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
33969,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
33970,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
33971,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
33972,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [28]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Classical Music'].index, size=20, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
33949,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
33950,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
33951,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
33952,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [29]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Ambient, Chill Filmscore'].index, size=420, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
33529,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
33530,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
33531,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
33532,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [30]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Others'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
33149,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
33150,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
33151,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
33152,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [31]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' World Music'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
32769,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
32770,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
32771,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
32772,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [32]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Hip Hop, Rap'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
32389,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
32390,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
32391,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
32392,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [33]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Electronica Filmscore'].index, size=360, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
32029,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
32030,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
32031,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
32032,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [34]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Filmscore Others'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
31649,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
31650,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
31651,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
31652,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [35]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Ambient, Chill Atmosphere'].index, size=250, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
31399,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
31400,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
31401,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
31402,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [36]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Pop Rock'].index, size=240, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
31159,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
31160,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
31161,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
31162,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [37]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Ambient, Chill'].index, size=220, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
30939,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
30940,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
30941,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
30942,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [40]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Country, Folk'].index, size=150, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
30789,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
30790,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
30791,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
30792,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [41]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Filmscore Pop'].index, size=250, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

,Audio_Filenames,track_genre_1,track_genre_2,length,multi_genres
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music,46.315102,Atmosphere World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music,96.600000,Atmosphere World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music,83.478277,Atmosphere World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music,101.329796,Atmosphere World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music,93.829773,Atmosphere World Music
...,...,...,...,...,...
30539,wpm099_10_Survival_745423.wav,Others,NaN,172.384943,Others
30540,wpm100_01_Help-Me-My-People_745439.wav,Rock,NaN,160.211882,Rock
30541,wpm100_03_Watching-Them-Falling_745441.wav,Rock,NaN,132.467891,Rock
30542,wpm100_04_Running_745442.wav,Rock,NaN,176.461519,Rock


In [42]:
audio_genres.to_csv('clean_audio_genres.csv', sep=';')